# Capacitated EOQ Model

## Model Inputs

Feel free to modify these to understand how the model works. Then `Run All`. 

In [ ]:
#This cell is for reading the system-wide inputs

total_hours = 8736
avail_regular_hours = 4264
avail_time_constraint = avail_regular_hours / total_hours
nominal_set_up_time = 10  #later we'll change this to a calculation
inv_holding_cost_percent = .2
cost_of_set_up_hour = 100


In [ ]:
#This cell is for reading the product-specific inputs


## Functions

In [ ]:
# All the functions we plan on using

## Model

In [ ]:
# Here's where the model runs

## Results

In [ ]:
# And here are the results!